In [22]:
pip install wikipedia-api

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import wikipedia

In [4]:
df = pd.read_csv('/content/Crop_recommendation.csv')

In [7]:
X = df[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = df['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
accuracy = model.score(X_test, y_test)
print('Model Accuracy:', accuracy)

Model Accuracy: 0.9454545454545454


In [35]:
import wikipediaapi
def recommend_crops(input_features, top_k=5):
    crop_predictions = model.predict(input_features)
    unique_crops = list(set(crop_predictions))
    crop_info = []
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI,
        user_agent='MyCropRecommendationApp/1.0'
    )

    for crop in unique_crops:
        try:
            page_py = wiki_wiki.page(crop)
            crop_summary = page_py.text[:500]
            crop_info.append((crop, crop_summary))
        except Exception as e:
            print(f"Failed to retrieve information for {crop}: {str(e)}")

    return crop_info[:top_k]

In [34]:
input_features = [[0.5, 0.2, 0.3, 25, 80, 6.5, 100]]
recommended_crops = recommend_crops(input_features)
for crop, summary in recommended_crops:
    print('Crop:', crop)
    print('Summary:', summary)
    print('---')

Crop: orange
Summary: Orange most often refers to:

Orange (fruit), the fruit of the tree species  Citrus × sinensis
Orange blossom, its fragrant flower
Orange (colour), from the color of an orange, occurs between red and yellow in the visible spectrum
Some other citrus or citrus-like fruit, see list of plants known as orange
Orange (word), both a noun and an adjective in the English languageOrange may also refer to:

Arts, entertainment, and media
Films
Game of Life (film), a 2007 film originally known as Oranges
Or
---


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [16]:
import pickle
filename = 'crop_recommendation_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)